In [35]:
# not using 
''' 

import pandas as pd
import numpy as np

#read database file used in 7-Dedup.
final = pd.read_csv("inputs/deduplicated_PC.csv", low_memory=False)

#read dedupe file
#df = pd.read_csv("outputs/probable_duplicates_CSD.csv")
df = pd.read_csv("outputs/pairs_CSD.csv")


def hierarchy(id1, id2): 

    try:

        check_beds1 = final.loc[final.idx == id1].number_beds.item()
        check_beds2 = final.loc[final.idx == id2].number_beds.item()
        check_type1 = final.loc[final.idx == id1].facility_type.item()
        check_type2 = final.loc[final.idx == id2].facility_type.item()
        check_file1 = final.loc[final.idx == id1].filename.item()
        check_file2 = final.loc[final.idx == id2].filename.item()
        check_name1 = final.loc[final.idx == id1].facility_name.item()
        check_name2 = final.loc[final.idx == id2].facility_name.item()

        filename_priority = ['QC_healthcare_facilities.csv', 'QC_Gatineau_public_places_including_hospitals,csv', 'QC_Quebec_City_public_places.csv', 
                             'ON_Ottawa_hospital.csv', 'ON_Durham_healthcare_facilities.csv', 'ON_healthcare_facilities.csv', 'NB_hospital_horizon.csv', 'NB_nursing_homes.csv']

        facilityname_priority = ['hospital', 'hôpita','hopita']

        if filename_priority.count(check_file1) == 1:

            if filename_priority.count(check_file2) == 1:

                priority1 = filename_priority.index(check_file1)
                priority2 = filename_priority.index(check_file2)

                if priority1 > priority2:
                    return id1
                elif priority1 < priority2:
                    return id2
                elif priority1 == priority2:

                    if any(substring in check_name1.lower() for substring in facilityname_priority):
                        return id2
                    elif any(substring in check_name2.lower() for substring in facilityname_priority):
                        return id1
                    else:
                        return id2

            elif filename_priority.count(check_file2) == 0:
                return id2

        elif filename_priority.count(check_file1) == 0:

            if filename_priority.count(check_file2) == 1:
                return id1

            elif filename_priority.count(check_file2) == 0:

                if check_file1 == 'CIHI_healthcare_facilities.csv' and check_file2 != 'CIHI_healthcare_facilities.csv':
                    return id1

                elif check_file2 == 'CIHI_healthcare_facilities.csv' and check_file1 != 'CIHI_healthcare_facilities.csv':
                    return id2

                else:

                    if check_beds1 == np.nan and check_beds2 == np.nan:

                        if any(substring in check_name1.lower() for substring in facilityname_priority):
                            return id2
                        elif any(substring in check_name2.lower() for substring in facilityname_priority):
                            return id1
                        else:
                            return id2

                    elif check_beds1 == np.nan and check_beds2 != np.nan:
                        return id1

                    elif check_beds1 != np.nan and check_beds2 == np.nan:
                        return id2

                    elif check_beds1 != np.nan and check_beds2 != np.nan:

                        if any(substring in check_name1.lower() for substring in facilityname_priority):
                            return id2
                        elif any(substring in check_name2.lower() for substring in facilityname_priority):
                            return id1
                        else:
                            return id2



    except ValueError:
        return "Item not found."


df["to_remove"] = df.apply(lambda x: hierarchy( id1 = x.idx1, id2 = x.idx2), axis = 1)
#df.to_csv('test.csv')
'''

In [22]:

import pandas as pd
import numpy as np

#read database file used in 7-Dedup.
final = pd.read_csv("inputs/deduplicated_PC.csv", low_memory=False)

#read dedupe file
#df = pd.read_csv("outputs/probable_duplicates_CSD.csv")
df = pd.read_csv("outputs/pairs_CSD.csv", low_memory=False)


def check_1(address, distance, name, streetnumber):

    if distance != None:
        if address >= 0.75 and distance < 2 and name > 0.7 and streetnumber == 1:
            return True
        else:
            return False

    else:
        return False

df["Check_1"] = df.apply(lambda x: check_1(address = x.Addr_CS, 
                          distance = x.Distance,
                          name = x.Name_CS,
                          streetnumber = x.StrNum_Match), axis=1)

#df.to_csv('test.csv')

In [23]:
def check_2(address, distance, name):

    if distance != None:
        if address >= 0.75 and distance < 1 and name > 0.85:
            return True
        else:
            return False

    else:
        return False

df["Check_2"] = df.apply(lambda x: check_2(address = x.Addr_CS, 
                          distance = x.Distance,
                          name = x.Name_CS), axis=1)


In [27]:
def check_3(address, distance, file):

    if distance != None:
        if address >= 0.99 and distance <= 0.5 and file == 1:
            return True
        else:
            return False

    else:
        return False

df["Check_3"] = df.apply(lambda x: check_3(address = x.Addr_CS, 
                          distance = x.Distance,
                          file = x.File_Match), axis=1)

In [28]:
def check_4(address, distance, streetnumber):


    if distance != None:
        if address >= 0.8 and distance < 0.1 and streetnumber == 1:
            return True
        else:
            return False

    else:
        return False

df["Check_4"] = df.apply(lambda x: check_4(address = x.Addr_CS, 
                          distance = x.Distance, 
                          streetnumber = x.StrNum_Match), axis=1)

In [29]:
def check_5(address, distance, name, streetnumber):

    if distance != None:
        if address >= 0.90 and distance < 2 and name > 0.93 and streetnumber == 1:
            return True
        else:
            return False

    else:
        return False

df["Check_5"] = df.apply(lambda x: check_5(address = x.Addr_CS, 
                          distance = x.Distance,
                          name = x.Name_CS,
                          streetnumber = x.StrNum_Match), axis=1)

In [30]:
# remove duplicates

rows = df.index[(df['Check_1'] == True) | (df['Check_2'] == True) | (df['Check_3'] == True) | (df['Check_4'] == True) | (df['Check_5'] == True)].tolist()
false_rows = df.index[(df['Check_1'] == False) & (df['Check_2'] == False) & (df['Check_3'] == False) & (df['Check_4'] == False) & (df['Check_5'] == False)].tolist()
print(rows)

df_true = df.loc[rows]
df_false = df.loc[false_rows]

df_true.to_csv('outputs/duplicates_CSD.csv', index=False)
df_false.to_csv('outputs/NOT_duplicates_CSD.csv', index=False)
#print(df_true)

hashes = df_true #['to_remove'].tolist()
#print(hashes)

final = final [~final['idx'].isin(hashes)]
final.to_csv('deduplicated_CSD.csv', index=False)
final.to_csv('inputs/deduplicated_CSD.csv', index=False)



[976, 1305, 1702, 1769, 2215, 2222, 2357, 2521, 2565, 3104, 3106, 3108, 3146, 3149, 3239, 3262, 3267, 3297, 3314, 3318, 3345, 3368, 3374, 3432, 3486, 3519, 3608, 3613, 3616, 3622, 3633, 3706, 3707, 3708, 3743, 3747, 3801, 3813, 3870, 3884, 3886, 3893, 3979, 3982, 4049, 4111, 4126, 4237, 4248, 4409, 4410, 4417, 4465, 4486, 4501, 4506, 4520, 4623, 4668, 4732, 4788, 4798, 4883, 4910, 4956, 4975, 5039, 5049, 5086, 5093, 5157, 5208, 5237, 5312, 5314, 5323, 5344, 5361, 5408, 5425, 5446, 5548, 5589, 5594, 5634, 5648, 5738, 5769, 5806, 5811, 5934, 6054, 6055, 6236, 6281, 6317, 6343, 6361, 6363, 6368, 6468, 6611, 6618, 6659, 6668, 6682, 6690, 6701, 6715, 6770, 6792, 6820, 6849, 6904, 6919, 6921, 6923, 6927, 6941, 7000, 7002, 7087, 7095, 7098, 7106, 7160, 7161, 7167, 7168, 7172, 7252, 7266, 7268, 7394, 7470, 7518, 7554, 7605, 7634, 7639, 7644, 7647, 7749, 7752, 7761, 7829, 7833, 7851, 8001, 8014, 8301, 8406, 8411, 8414, 8439, 8447, 8477, 8487, 8600, 8620, 8631, 8634, 8643, 8801, 8802, 8808, 8815